In [133]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
import json
from inspect import signature
from pydantic import BaseModel, create_model
from typing import List, Any, Callable



class Function(BaseModel):
    def __init__(self, func: Callable[..., Any]):
        super().__init__()
        self._func = func
        self._name = func.__name__
        self._doc = func.__doc__
        self._signature = signature(func)
        self._return_type = self._signature.return_annotation

    def name(self):
        return self._name

    def doc(self):
        return self._doc

    def run(self, args: Any) -> Any:
        # If args is not already a BaseModel, attempt to create a dynamic model.
        if not isinstance(args, BaseModel):
            # Create a model dynamically based on the function's signature.
            model_cls = create_model('DynamicArgs', **{k: (v.annotation, ...) for k, v in self._signature.parameters.items()})
            args = model_cls(**args)
        
        # Call the function with unpacked arguments from the BaseModel.
        result = self._func(**args.model_dump())
        
        # Check if the result matches the expected return type.
        assert self._return_type == type(result), f"Expected return type {self._return_type}, but got {type(result)}"
        return result

class Tool(Function):
    def __init__(self, func: Callable[..., Any]):
        super().__init__(func)
    
    @property
    def schema(self):
        schema = {
            "name": self.name(),
            "doc": self.doc(),
            "parameters": {
                "properties": {
                    param_name: {"type": param.annotation.__name__ if param.annotation != param.empty else "unknown"}
                    for param_name, param in self._signature.parameters.items()
                }
            }
        }
    
        return schema

class ToolKit(BaseModel):
    def __init__(self, tools: List[Tool]):
        super().__init__()
        # Store each function as a Tool instance
        self._tools = tools
        
    def tools(self):
        return self._tools


def test(a: int, b: List = [1,2,3]) -> list:
    return a * b

def test2(c: str, d: int) -> str:
    return c*d

tool = Tool(test)
result = tool.run({'a': 1, 'b': [1, 2]})
result

[1, 2]

In [142]:
tools = ToolKit([test, test2])

In [147]:
tools.tools()

[<function __main__.test(a: int, b: List = [1, 2, 3]) -> list>,
 <function __main__.test2(c: str, d: int) -> str>]

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [139]:
function.name()

'test'

In [123]:
sdf

NameError: name 'sdf' is not defined

In [13]:
sig = signature(test)

In [23]:
for param_name, param in sig.parameters.items():
    print(param_name, param.annotation)

a <class 'str'>


In [15]:
sig.return_annotation

str